# Segment raw images to get gray matter before normalising.
This is to prevent errors in normalisation down the line (e.g. enlargement of ventricles: GM WM growth or shrinkage?)

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
import numpy as np
import os

200109-12:47:27,975 nipype.utils INFO:
	 Running nipype version 1.4.0-dev (latest: 1.4.0)


In [2]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=list()
    for i in os.listdir(dir):
        if os.path.isdir(os.path.join(dir, i)):
            subdirs.append(os.path.join(dir, i))
    return sorted(subdirs)

In [3]:
subdirs=get_subdirs("/VBM/")
print(subdirs)

['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED011', '/VBM/FED012', '/VBM/FED013', '/VBM/FED014', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED018', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED050', '/VBM/FED051', '/VBM/FED052', '/VBM/FED053', '/VBM/FED054', '/VBM/FED055', '/VBM/FED056', '/VBM/FED057', '/VBM/FED058', '/VBM/FED059', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED066', '/VBM/FED067', '/VBM/FED068']


In [4]:
# make dict with all images as path references
images=dict()
for i in subdirs:
    for j in os.listdir(i):
        if j.endswith(".nii"):
            images[i.rsplit('/', 1)[1]]=os.path.join(i, j)

print(images)

# load these in different DATA FORMATS (nibabel, pandas) later on

{'FED006': '/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii', 'FED007': '/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii', 'FED008': '/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii', 'FED009': '/VBM/FED009/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii', 'FED010': '/VBM/FED010/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141119091904_5.nii', 'FED011': '/VBM/FED011/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141119161708_5.nii', 'FED012': '/VBM/FED012/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141124150148_5.nii', 'FED013': '/VBM/FED013/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141125093955_5.nii', 'FED014': '/VBM/FED014/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141125153216_5.nii', 'FED015': '/VBM/FED015/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141202145526_5.nii', 'FED016': '/VBM/FED016/T1_MPRAGE_SAG_ISO_0_9_0005

In [8]:
# use NewSegment from spm to segment unimportant anatomical landmarks and perform "brain-extraction" (TEST FED006)
#for i in images.keys(): use this when ready to segment full subject list

# classify tissues from tpm.nii
# need to perform segmentation with full tissue information, or brain will be classified as "soft tissue" etc. -> see examples in "/VBM/FED006/"
tissue1 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 1), 2, (True,True), (False, False)) # GM
tissue2 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 2), 2, (True,True), (False, False)) # WM
tissue3 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 3), 2, (True,True), (False, False)) # CSF
tissue4 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 4), 2, (False,False), (False, False)) # bone
tissue5 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 5), 2, (False,False), (False, False)) # soft tissue
tissue6 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 6), 2, (False,False), (False, False)) # air and background

# create SKULLSTRIP segment nodes for each subject
for i in images2.keys():
    segm_skullstrip_T1=Node(spm.preprocess.NewSegment(), name='segm_skullstrip_T1')
    segm_skullstrip_T1.inputs.tissues=[tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
    # do not generate SPM8 and higher compatible jobs
    segm_skullstrip_T1.inputs.use_v8struct=False
    segm_skullstrip_T1.inputs.affine_regularization='none'
    # write forward and inverse deformation fields
    #segm_skullstrip_T1.inputs.write_deformation_fields=[True, True]
    # establish input files from subject
    segm_skullstrip_T1.inputs.channel_files=images2[i]
    # establish basedir for output of subject node
    for j in subdirs:
        if j.rsplit('/', 1)[1]==i:
            segm_skullstrip_T1.base_dir=j
    results=segm_skullstrip_T1.run()
    print(results.outputs)

200109-12:58:20,555 nipype.workflow INFO:
	 [Node] Setting-up "segm_skullstrip_T1" in "/VBM/FED009/segm_skullstrip_T1".
200109-12:58:20,562 nipype.workflow INFO:
	 [Node] Running "segm_skullstrip_T1" ("nipype.interfaces.spm.preprocess.NewSegment")
200109-13:02:05,551 nipype.workflow INFO:
	 [Node] Finished "segm_skullstrip_T1".

bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = [['/VBM/FED009/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii'], ['/VBM/FED009/segm_skullstrip_T1/rc2T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii'], ['/VBM/FED009/segm_skullstrip_T1/rc3T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii'], [], [], []]
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = [[], [], [], [], [], []]
native_class_images = [['/VBM/FED009/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20

In [ ]:
# write the same thing for the next 3 subjects 
# using datasink and iterables nodes -> better workflow visualisation




In [12]:
# crop down images for testing (only FED006,07,08!!!)
images2=dict()
for i in sorted(images.keys())[0:3]:
    images2[i]=images[i]
    
print(images2.items())

dict_items([('FED006', '/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'), ('FED007', '/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii'), ('FED008', '/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii')])


In [13]:

import nibabel as nib
import re

segmdirs=dict()

# define start of segmentation directory name
targetnode="segm"

for i in images2.keys():
    for j in subdirs:
        # match keys with subdir list
        if j.rsplit('/',1)[1] == i: 
            for k in os.listdir(j):
                # go through content of subdirs and look for "true" directories
                if os.path.isdir(os.path.join(j, k)):
                    # regex functions match=exact match of pattern, search=finds elements that contain pattern
                    # regex matches entries in j with targetnode k
                    match=re.match(r'(' + targetnode + ').*', k)
                    # ? This is needed to recognize the attribute group from the regex output
                    if match: 
                        segmdirs[j]=os.path.join(j, match.group(0))

In [14]:
# transform posthoc probability maps for bone and soft tissues into nibabel objects

posttpms=dict()

# enter tissue class name from segment.output=native-space drawn from TPM.nii classifications
targettp=["c1T1", "c2T1", "c3T1"]
for i in segmdirs.keys():
    # prepare list for elements in posttpms
    objects=list()
    for j in os.listdir(segmdirs[i]):
        for t in targettp:
            # match entries in j with targettp in t
            match=re.match(r'(' + t + ').*', j)
            # appends to the objects list, if match is found
            if match:
                objects.append(nib.load(os.path.join(segmdirs[i], match.group(0))))
    # connects the objects list to subjects entry in posttpms dict
    posttpms[i]=objects
        
# explore posttpms
print(posttpms.items())

dict_items([('/VBM/FED006', [<nibabel.nifti1.Nifti1Image object at 0x7ff2a80249b0>, <nibabel.nifti1.Nifti1Image object at 0x7ff2a8024b70>, <nibabel.nifti1.Nifti1Image object at 0x7ff2a8024cf8>]), ('/VBM/FED007', [<nibabel.nifti1.Nifti1Image object at 0x7ff2a8024e80>, <nibabel.nifti1.Nifti1Image object at 0x7ff2a80248d0>, <nibabel.nifti1.Nifti1Image object at 0x7ff2a80271d0>]), ('/VBM/FED008', [<nibabel.nifti1.Nifti1Image object at 0x7ff2a8027358>, <nibabel.nifti1.Nifti1Image object at 0x7ff2a80274e0>, <nibabel.nifti1.Nifti1Image object at 0x7ff2a83a2828>])])


In [19]:
from nipype.interfaces.fsl import ImageMaths

# prepare list for elements in posttpms
segm_GM=list()

for i in segmdirs.keys():
    print(i)
    # get structural image
    for j in os.listdir(i):
        a1=re.match(r'.*(mprage).*.nii', j)
        if a1:
            a=os.path.join(i, a1.group(0))
    # get posttpm (b)
    for k in os.listdir(segmdirs[i]):
        b1=re.match(r'(^c1T1).*.nii', k)
        if b1:
            b=os.path.join(segmdirs[i], b1.group(0))
    print(a,b)
    c=ImageMaths(in_file=a, op_string="-mul b", out_file="segm-GM-structural")
    segm_GM.append(c)

/VBM/FED006
/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii /VBM/FED006/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii
/VBM/FED007
/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii /VBM/FED007/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii
/VBM/FED008
/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii /VBM/FED008/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii


In [61]:
%%bash
# check if operator -mul is even appropriate in this case
fslmaths -h


Usage: fslmaths [-dt <datatype>] <first_input> [operations and inputs] <output> [-odt <datatype>]

Datatype information:
 -dt sets the datatype used internally for calculations (default float for all except double images)
 -odt sets the output datatype ( default is float )
 Possible datatypes are: char short int float double input
 "input" will set the datatype to that of the original image

Binary operations:
  (some inputs can be either an image or a number)
 -add   : add following input to current image
 -sub   : subtract following input from current image
 -mul   : multiply current image by following input
 -div   : divide current image by following input
 -rem   : modulus remainder - divide current image by following input and take remainder
 -mas   : use (following image>0) to mask current image
 -thr   : use following number to threshold current image (zero anything below the number)
 -thrp  : use following percentage (0-100) of ROBUST RANGE to threshold current image (zero any

Create the study specific, non-linear GM template